In [1]:
# models
from src.models.model_util import load_model, save_checkpoint

# Pytorch Lightning
import pytorch_lightning as pl
from src.lightning.lit_model import *
from src.lightning.custom_callbacks import *

# datetime
from datetime import datetime

# other files
from src.training.run import *
from src.config import *
from src.data.data_util import *

In [2]:
cfg = load_config()
cfg.model_name = 'vanilla'

In [3]:
cfg.test = False
trn_dset = MyDataset(cfg, augment=None)
trn_dl = DataLoader(trn_dset, cfg.batch_size)
cfg.test = True
val_dset = MyDataset(cfg, augment=None)
val_dl = DataLoader(val_dset, cfg.batch_size)

In [4]:
today = datetime.today()
fmt = fmt = lambda t: str(t).zfill(2)
today = f'{today.year}.{fmt(today.month)}.{fmt(today.day)}.{fmt(today.hour)}:{fmt(today.minute)}'

loggers = [pl.loggers.MLFlowLogger(save_dir="result/mlruns", experiment_name=cfg.model_name)]
callbacks = [pl.callbacks.EarlyStopping(monitor="val_acc", mode="min", patience=11*5),]
#              pl.callbacks.ModelCheckpoint("models/", monitor="val_acc", prefix='resnet_conv1_'),]
#              PrintCallback(cfg)]

model, cfg.device = load_model(cfg.model_name, cfg=cfg, gpu=False, verbose=False)
lit_model = BaseLitModel(model, cfg=cfg)

Model Vanilla is selected.


In [5]:
trainer = pl.Trainer(gpus=1, logger=loggers, callbacks=callbacks)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [6]:
# trainer.tune(lit_model, train_dataloader=trn_dl, val_dataloaders=val_dl)

In [7]:
trainer.fit(lit_model, train_dataloader=trn_dl, val_dataloaders=val_dl)


  | Name      | Type              | Params
------------------------------------------------
0 | model     | Vanilla3d         | 18.2 K
1 | train_acc | MeanAbsoluteError | 0     
2 | val_acc   | MeanAbsoluteError | 0     
3 | test_acc  | MeanAbsoluteError | 0     
------------------------------------------------
18.2 K    Trainable params
0         Non-trainable params
18.2 K    Total params
0.073     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

RuntimeError: The size of tensor a (23) must match the size of tensor b (96) at non-singleton dimension 4

`trainer._results` 확인해보기   
GradCAM 완전 독립으로 만들어서 -> naive pytorch 적용하고 -> 그 다음에 PL 넣기   
Checkpoints in callbacks 모듈 확인